In [1]:
import pandas as pd
from lightgbm import LGBMRegressor
import optuna
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization import plot_optimization_history
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,\
mean_squared_log_error
from catboost import CatBoostRegressor
import shap
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
import warnings
warnings. filterwarnings("ignore")
RAND = 10
N_FOLDS = 3

In [2]:
def r2_adjusted(y_true: np.ndarray, y_pred: np.ndarray,X_test: np.ndarray) -> float:
    """Коэффициент детерминации (множественная регрессия)"""
    N_objects = len(y_true)
    N_features = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)

def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean percentage error"""
    return np.mean((y_true - y_pred) / y_true) * 100

def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean absolute percentage error"""
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100
def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Weighted Absolute Percent Error"""
    return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100
def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    """The Root Mean Squared Log Error (RMSE) metric
    Логарифмическая ошибка средней квадратичной ошибки"""
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except:
        return None
def get_metrics(y_test: np.ndarray,y_pred: np.ndarray,X_test: np.ndarray,name: str = None):
    """Генерация таблицы с метриками"""
    df_metrics = pd.DataFrame()
    df_metrics['model']=[name]
    df_metrics['MAE'] = mean_absolute_error(y_test, y_pred)
    df_metrics['MSE'] = mean_squared_error(y_test, y_pred)
    df_metrics['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
    df_metrics['RMSE'] = rmse(y_test, y_pred)
    df_metrics['R2 adjusted'] = r2_adjusted(y_test, y_pred,X_test)
    df_metrics['MPE_%'] = mpe(y_test, y_pred)
    df_metrics['MAPS_%'] = mapper(y_test, y_pred)
    df_metrics['WAPE_%'] = wape(y_test, y_pred)
    
    return df_metrics
    
def check_overfitting(model, X_train, y_train, X_test, y_test, metric_fun):
    """Проверка на overfitting"""
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    may_train = metric_fun(y_train, y_pred_train)
    may_test = metric_fun(y_test, y_pred_test)
    print(f'{metric_fun._name_} train: %.3f' % mae_train)
    print(f'{metric_fun._name_} test: %.3f' % mae_test)
    print(f'delta = {(abs(made_trading - make_test)/make_train*100):.1f} %')

In [5]:
df = pd.read_excel('clear_data.xlsx')
df = df.drop('Номер объявления', axis = 1)

In [6]:
X = df.drop(['Цена'], axis=1) 
y = df['Цена']


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=101,
                                                    shuffle=True)

X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.16,
                                                  random_state=101,
                                                  shuffle=True)

eval_set = [(X_val, y_val)]
from sklearn.compose import ColumnTransformer
transformers_list = [
                    ('encode', OneHotEncoder(dtype='int', drop='first'), [
                    'Тип продавца',
                    'Район',
                    'Санузел', 'Ремонт', 'Вид сделки'
                    ]),
                    ('scale', StandardScaler(),
                    ['Количество комнат', 'Общая площадь', 'Этаж', 'Этажей в доме',  'Пассажирский лифт'])]
column_transformer = ColumnTransformer(transformers_list)

pipe = Pipeline([('columnTransformer', column_transformer),
                ('lr', LinearRegression())])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

ValueError: Found unknown categories ['Аукцион'] in column 4 during transform